# Heritage Housing Data Cleaning

## Objectives
- Load the raw Ames Housing dataset
- Clean missing values and drop irrelevant features
- Prepare the data for analysis and modeling

## Inputs
- data/raw/house_prices_records.csv

## Outputs
- data/processed/cleaned_data.csv

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 


## Check for Missing Values
We begin identifying which columns have missing data, so we can decide how to clean them.

In [2]:
import pandas as pd

# Load the dataset
df = pd.read_csv("../data/raw/house_prices_records.csv")

# Display the first few rows
df.head()

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,856,854.0,3.0,No,706,GLQ,150,0.0,548,RFn,...,65.0,196.0,61,5,7,856,0.0,2003,2003,208500
1,1262,0.0,3.0,Gd,978,ALQ,284,NaN,460,RFn,...,80.0,0.0,0,8,6,1262,NaN,1976,1976,181500
2,920,866.0,3.0,Mn,486,GLQ,434,0.0,608,RFn,...,68.0,162.0,42,5,7,920,NaN,2001,2002,223500
3,961,NaN,NaN,No,216,ALQ,540,NaN,642,Unf,...,60.0,0.0,35,5,7,756,NaN,1915,1970,140000
4,1145,NaN,4.0,Av,655,GLQ,490,0.0,836,RFn,...,84.0,350.0,84,5,8,1145,NaN,2000,2000,250000


In [3]:
# Show missing values in the dataset
missing = df.isnull().sum()
missing[missing > 0].sort_values(ascending=False)

EnclosedPorch    1324
WoodDeckSF       1305
LotFrontage       259
GarageFinish      235
BsmtFinType1      145
BedroomAbvGr       99
2ndFlrSF           86
GarageYrBlt        81
BsmtExposure       38
MasVnrArea          8
dtype: int64

### Clean Missing Values 

Clean the dataset by handling missing values.
Drop columns with too many missing entries and fill the rest with sensible defaults

Improves the data quality which is essential for accurate modeling
Makes sure the algorithms won't crash or give poor results

In [4]:
# Drop columns with too many missing values 
df = df.drop(columns=["EnclosedPorch", "WoodDeckSF", "LotFrontage"])

# Fill missing values for categorical columns with 'None'
df["GarageFinish"] = df["GarageFinish"].fillna("None")
df["BsmtFinType1"] = df["BsmtFinType1"].fillna("None")

# Fill missing values for numerical columns with the median value
df["BedroomAbvGr"] = df["BedroomAbvGr"].fillna(df["BedroomAbvGr"].median())
df["GarageYrBlt"] = df["GarageYrBlt"].fillna(df["GarageYrBlt"].median())

# Check remaining missing values in the dataset
df.isnull().sum().sort_values(ascending=False).head(10)

2ndFlrSF        86
BsmtExposure    38
MasVnrArea       8
BedroomAbvGr     0
1stFlrSF         0
BsmtFinSF1       0
BsmtFinType1     0
GarageArea       0
BsmtUnfSF        0
GarageYrBlt      0
dtype: int64

### Handle Remaining Missing Values 

Fill in the remaining missing values using appropriate strategies:
- **Numerical columns** are filled with the **median**, which is robust to outliers.
- **Categorical columns** (like 'BsmtExposure') are filled with "None" to preserve the structure of the data.
This step is critical to ensure the dataset is complete and safe for modeling.

In [5]:
# Fill missing numerical columns with median
df["2ndFlrSF"] = df["2ndFlrSF"].fillna(df["2ndFlrSF"].median())
df["1stFlrSF"] = df["1stFlrSF"].fillna(df["1stFlrSF"].median())
df["BsmtExposure"] = df["BsmtExposure"].fillna("None")
df["MasVnrArea"] = df["MasVnrArea"].fillna(df["MasVnrArea"].median())
df["GarageArea"] = df["GarageArea"].fillna(df["GarageArea"].median())
df["BsmtFinSF1"] = df["BsmtFinSF1"].fillna(df["BsmtFinSF1"].median())

# Final check for any remaining missing values
df.isnull().sum().sort_values(ascending=False).head(10)

1stFlrSF        0
2ndFlrSF        0
BedroomAbvGr    0
BsmtExposure    0
BsmtFinSF1      0
BsmtFinType1    0
BsmtUnfSF       0
GarageArea      0
GarageFinish    0
GarageYrBlt     0
dtype: int64

In [9]:
# Save the cleaned dataset to a new CSV file in the processed folder
# We use index=False to avoid saving the row numbers as an extra column

df.to_csv("../data/processed/cleaned_data.csv", index=False)